<a href="https://colab.research.google.com/github/marcelogarret/reconpad/blob/main/aula5a_gridsearch_garret_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Importando o dataset diabetes (regressão)
from sklearn.datasets import load_diabetes
X, y = load_diabetes(return_X_y=True)
X.shape, y.shape

((442, 10), (442,))

In [2]:
## !!!Solução Principal!!! ##
## Importando bibliotecas de seleção de modelos, métricas, pré-processamento e pipeline
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_validate, RepeatedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np

## Construindo o Pipeline com o StandardScaler e o modelo de regressão Knn
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsRegressor())
])

## Definindo parãmetros de ajuste do GridSearchCV
param = {'scaler__with_mean': [True, False],
         'clf__n_neighbors': [3, 5, 7],
         'clf__metric': ['cosine', 'cityblock', 'euclidean']}

## Definindo modelo aninhado com Pipeline interno ao GridSearchcV
modeloAninhado = GridSearchCV(pipe, param, verbose=0, cv=5,
                      scoring=make_scorer(mean_squared_error,
                                          greater_is_better=False,
                                          squared=False))

## Realizando a validação cruzada com 5 folds e retornando o resultado do treinamento
scores = cross_validate(modeloAninhado, X, y, cv=5, return_estimator=True,
                        scoring=make_scorer(mean_squared_error, squared=False), return_train_score=True)

## Comparando o resultado médio dos treinamentos e dos testes da validação cruzada
print(f"RMSE Treinamento = {np.mean(scores['train_score'])}\nRMSE Teste = {np.mean(scores['test_score'])}")

RMSE Treinamento = 50.75526155219973
RMSE Teste = 57.27230638368057


In [3]:
## !!!Solução Alternativa!!! ##
## Dividindo o dataset em treino e teste
from sklearn.model_selection import train_test_split
Xtr, Xte, ytr, yte = train_test_split(X, y)

## Construindo o Pipeline com o StandardScaler e o modelo de regressão Knn
pipe2 = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', KNeighborsRegressor())
])

## Definindo parãmetros de ajuste do GridSearchCV
param2 = {'scaler__with_mean': [True, False],
         'clf__n_neighbors': [3, 5, 7],
         'clf__metric': ['cosine', 'cityblock', 'euclidean']}

## Definindo modelo aninhado com Pipeline interno ao GridSearchcV
modeloAninhado2 = GridSearchCV(pipe2, param2, verbose=0, cv=5,
                      scoring=make_scorer(mean_squared_error,
                                          greater_is_better=False,
                                          squared=False))

## Realizando a validação cruzada com 5 folds e retornando o resultado do treinamento
scores2 = cross_validate(modeloAninhado2, X, y, cv=5, return_estimator=True,
                        scoring=make_scorer(mean_squared_error, squared=False), return_train_score=True)

## Comparando o resultado médio do treinamento e do teste
rmse_train = 0
rmse_test = 0
for modelo in scores2['estimator']:
  rmse_train += mean_squared_error(ytr, modelo.predict(Xtr))**0.5
  rmse_test += mean_squared_error(yte, modelo.predict(Xte))**0.5
print(f"RMSE Treinamento = {rmse_train/len(scores2['estimator'])}\nRMSE Teste = {rmse_test/len(scores2['estimator'])}")

RMSE Treinamento = 53.25681387427882
RMSE Teste = 48.592116401560126
